In [1]:
from math import sqrt, ceil
import numpy as np
from scipy.sparse.linalg import norm
import matplotlib.pyplot as plt
import pandas as pd
import cirq
import openfermion as of
from openfermionpyscf import run_pyscf
import qiskit
from qiskit.circuit.library import PauliEvolutionGate, phase_estimation
from qiskit.synthesis import LieTrotter
from qiskit import transpile
from qpe_trotter import get_gate_counts
from convert import cirq_pauli_sum_to_qiskit_pauli_op

In [2]:
def eps2_bound(hamiltonian: cirq.PauliSum) -> float:
    """Get a bound on eps2."""

    M = len(hamiltonian)
    coeffs = np.array([ps.coefficient for ps in hamiltonian])
    i_max = np.argmax(np.abs(coeffs))
    max_coeff = coeffs[i_max]
    n_summands = M
    return (-1. / 24) * 0.5 * max_coeff ** 3 * n_summands ** 3

In [12]:
def tau_bound(ham_cirq: cirq.PauliSum) -> float:
    """Get a lower bound on tau from the traingle inequality."""

    coeffs = np.array([ps.coefficient for ps in ham_cirq])
    ham_norm = np.sum(np.abs(coeffs))
    tau = np.pi / (4. * ham_norm)
    return tau

In [15]:
hamiltonian_threshold = 2e-2
energy_error = 1e-3

In [13]:
problem = "reactant"
data = np.load(f"./po3_1w_{problem}_22o.npz")
ecore = data["e_nuc"]
h1 = np.array(data["h1"])
h2 = np.array(data["h2"])
nelectrons = data["nelectron"]

h1_new, h2_new = of.chem.molecular_data.spinorb_from_spatial(h1, h2)
h = of.InteractionOperator(ecore.item(), h1_new, h2_new)
hamiltonian = of.get_fermion_operator(h)
if hamiltonian_threshold is not None:
    hamiltonian.compress(hamiltonian_threshold)
hamiltonian = of.jordan_wigner(hamiltonian)
ham_cirq = of.transforms.qubit_operator_to_pauli_sum(hamiltonian)

In [18]:
eps2 = eps2_bound(ham_cirq)
tau = tau_bound(ham_cirq)
dt = sqrt(energy_error / abs(eps2))
nsteps = ceil(tau / dt)
print(f"eps2 = {eps2:4.5e}")
print(f"dt = {dt:4.5e}")
print(f"tau = {tau:4.5e}")
print(f"steps = {nsteps}")

eps2 = 1.10783e+20+0.00000e+00j
dt = 3.00444e-12
tau = 6.57375e-04
steps = 218801062


In [19]:
problem = "product"
data = np.load(f"./po3_1w_{problem}_22o.npz")
ecore = data["e_nuc"]
h1 = np.array(data["h1"])
h2 = np.array(data["h2"])
nelectrons = data["nelectron"]

h1_new, h2_new = of.chem.molecular_data.spinorb_from_spatial(h1, h2)
h = of.InteractionOperator(ecore.item(), h1_new, h2_new)
hamiltonian = of.get_fermion_operator(h)
if hamiltonian_threshold is not None:
    hamiltonian.compress(hamiltonian_threshold)
hamiltonian = of.jordan_wigner(hamiltonian)
ham_cirq = of.transforms.qubit_operator_to_pauli_sum(hamiltonian)

In [20]:
eps2 = eps2_bound(ham_cirq)
tau = tau_bound(ham_cirq)
dt = sqrt(energy_error / abs(eps2))
nsteps = ceil(tau / dt)
print(f"eps2 = {eps2:4.5e}")
print(f"dt = {dt:4.5e}")
print(f"tau = {tau:4.5e}")
print(f"steps = {nsteps}")

eps2 = 1.21074e+20+0.00000e+00j
dt = 2.87391e-12
tau = 6.47921e-04
steps = 225448938
